In [11]:
!pip install datasets

In [12]:
# Install evaluate library
!pip install evaluate

In this project, I will demonstrate fine-tuning a GPT-2 model to enhance its ability to infer the sentiments of tweets. Given its limitations in this area, a natural question arises: To improve it's performance, I will leverage fine-tuning by training the pre-trained GPT-2 model from Hugging Face using a dataset of tweets paired with their corresponding sentiments. Below is a concise outline of the process:

Step 1: Select a Pre-Trained Model and Dataset
The first step in fine-tuning is to choose a pre-trained model. For this task, I will utilize GPT-2 as the base model, alongside a suitable dataset for effective fine-tuning.


Step 2: Load the Dataset
With the model selected, the next critical step is to acquire high-quality data for fine-tuning. This is where the Hugging Face Datasets library becomes invaluable.
For this project, I will utilize the Hugging Face Datasets library to import a dataset that categorizes tweets based on their sentiments specifically, Positive, Neutral, or Negative. This structured dataset will serve as the foundation for enhancing the model's sentiment analysis capabilities


In [13]:




from datasets import load_dataset
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

dataset = load_dataset("mteb/tweet_sentiment_extraction")

df = pd.DataFrame(dataset['train'])


Step 3: Tokenize the Dataset
Having acquired the dataset, the next essential step is to prepare it for the model by utilizing a tokenizer. Since large language models (LLMs) operate on tokens, a tokenizer is necessary to appropriately process the dataset.
To efficiently tokenize the entire dataset, I will employ the Datasets library's map method, which allows for the application of a preprocessing function across the complete dataset in one step. Therefore, the focus of this step is to load a pre-trained tokenizer and tokenize the dataset, ensuring it is ready for fine-tuning.

In [14]:
from transformers import GPT2Tokenizer
dataset = load_dataset("mteb/tweet_sentiment_extraction")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

To optimize our processing requirements, I will create a smaller subset of the full dataset for fine-tuning the model. This approach allows for more efficient training while maintaining effective evaluation.
The training set will be utilized for fine-tuning the model, whereas the testing set will serve as a benchmark for evaluating its performance. This division ensures that we can assess the model's capabilities on unseen data, enhancing the overall robustness of our fine-tuning process


In [15]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Step 4: Initialize the Base Model,The next step is to load the base model and configure it to recognize the specific number of labels relevant to this task. Based on the sentiment analysis dataset for tweets,  there are three distinct labels: Positive, Neutral, and Negative.
By initializing the model with this information, It is ensured that it is adequately prepared for the fine-tuning process and capable of accurately classifying the sentiment of the input data


In [16]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step 5: Define the Evaluation Method,
The Transformers library offers a Trainer class optimized for training models However, the default implementation does not include a mechanism for evaluating model performance. Therefore, prior to commencing the training process, an evaluation function will need to be provided to the Trainer.
This evaluation function will facilitate the assessment of the model's performance throughout the training process, ensuring that improvements can be monitored and quantified effectively.


In [17]:


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

Step 6: Fine-Tune Using the Trainer Method
The final step involves configuring the training arguments and initiating the training process. The Transformers library provides the Trainer class, which supports a comprehensive array of training options and features, including logging, gradient accumulation, and mixed precision.
Initially, the training arguments will be defined along with the evaluation strategy. Once all parameters are established, the model can be trained effortlessly using the train() command, streamlining the fine-tuning process.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir="test_trainer",

   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   gradient_accumulation_steps=4
   )


trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,

)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


Step,Training Loss


**Model Evaluation**

Following the training process, the model’s performance will be evaluated on a validation or test set. The Trainer class includes an evaluate method that efficiently manages this evaluation. By utilizing this built-in functionality, a comprehensive assessment of the model's performance can be conducted, providing valuable insights into its effectiveness and accuracy.



In [ ]:

trainer.evaluate()


These are the most basic steps to perform a fine-tuning of any LLM and fine-tuning an LLM is highly computationally demanding.